This script is used to perform lesion segmentation in fundus photographs. The lesions that can be segmented are hard exudates, soft exudates, microaneurysms and hemorrhages. The segmentation is based on a UNet, a CNN that takes an image as an input and that outputs a probability map indicating for every pixel the probability of belonging to a certain type of lesion or not.

In [ ]:
!pip install tensorlayer==1.11.1
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# import necessary libraries

import numpy as np
import tensorflow as tf
import tensorlayer as tl
import tensorlayer.layers as tll
import tensorflow.keras.backend as K

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import time

print(tf.__version__)
print(tl.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# read in train and test data in case Google DRIVE is used
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# read in the train and test data for a certain lesion type

# Basepath depends on the lesion
# LesionType = 'SoftExudates'
LesionType = 'HardExudates'
# LesionType = 'Microaneurysms'
# LesionType = 'Hemorrhages'

In [ ]:
# Basepath for Google DRIVE:
# Basepath = '/content/drive/My Drive/Stage_ENT_Studios/Data/IDRiD/' + LesionType + '/Arrays/'

# Basepath for Jupyter notebooks:
Basepath = 'C:/Users/lunam/Documents/1steMaster/Stage/Data_FinalArrays/IDRiD/'+ LesionType+'/Arrays/'

# train data
train_images = np.float32(np.load(Basepath + 'train_images_Final.npy'))
print('Shape train images: {}'.format(train_images.shape))

train_annotations =  np.int32(np.load(Basepath + 'train_annotations_Final.npy'))
# train_annotations = np.expand_dims(train_annotations, axis = 3)
print('Shape train annotations: {}'.format(train_annotations.shape))

# test data
test_images = np.float32(np.load(Basepath + 'test_images_Final.npy'))
print('Shape test images: {}'.format(test_images.shape))

test_annotations = np.int32(np.load(Basepath + 'test_annotations_Final.npy'))
# test_annotations = np.expand_dims(test_annotations, axis = 3)
print('Shape test annotations: {}'.format(test_annotations.shape))

Shape train images: (324, 512, 512, 3)
Shape train annotations: (324, 512, 512)
Shape test images: (156, 512, 512, 3)
Shape test annotations: (156, 512, 512)


In [ ]:
# save directions for the model and the tensorboard logs

# Base path for Google DRIVE:
# base_path = '/content/drive/My Drive/Stage_ENT_Studios/Unet/Logs/'

# Base path for jupyter notebooks:
base_path = 'C:/Users/lunam/Documents/1steMaster/Stage/Code_Final/DR_classification/FeatureBasedClassification/UNet_Sigmoid/Logs/'+LesionType+'/'

# direction where the tensorboard files will be stored
log_dir_tens = base_path + 'Tensorboard_Logs/'
# direction where the trained models will be stored
log_dir_model = base_path + 'Trained_Model/'

In [ ]:
class BatchData():
    '''
    This class is used to create batches of images with their corresponding annotations
    These batches can then be fed to the Unet
    The numpy arrays of preprocessed images and corresponding annotations are given as an input to the class
    '''
    
    # resetting all values in case a new batch dataset is created
    images = []
    annotations = []
    batch_offset = 0
    epochs_completed = 0 


    def __init__(self, image_arrays, annotation_arrays):
        
        print("Initializing Batch Dataset Reader...")
        
        self.images = image_arrays
        self.annotations = annotation_arrays

        # resize the annotations to indicate that there is only one channel in this case
        self.annotations = np.expand_dims(self.annotations, 3)
        # this makes a binary mask of the annotations, there where annotations has a pixel value above 0, the pixel value will be set to 1
        # there where the pixel value is 0 or smaller, the pixel value is set to 0
        self.annotations = np.where(self.annotations>0, 1, 0)
        
        print ('Shape images: {}'.format(self.images.shape))
        print ('Shape annotations: {}'.format(self.annotations.shape))
    
    def next_batch(self, batch_size):
        '''
        This function can be used to everytime find the next batch of images and corresponding annotations
        The size of the batches is defined by the batch_size
        '''
        
        # start image of the batch
        start = self.batch_offset
        # set the starting value for the next batch
        self.batch_offset += batch_size 
        
        # all data has already been used
        if self.batch_offset > self.images.shape[0]:
            
            # images and annotations get shuffled randomly for the next epoch
            perm = np.arange(self.images.shape[0])
            np.random.shuffle(perm)
            self.images = self.images[perm]
            self.annotations = self.annotations[perm]
            
            # Start next epoch
            start = 0
            self.batch_offset = batch_size
        
        # end image of the batch
        end = self.batch_offset
        
        # return the images and corresponding annotations in this batch
        return self.images[start:end], self.annotations[start:end]

In [ ]:
def LoadBatchData(Train_Images, Train_Annotations, Test_Images, Test_Annotations):
    '''
    This function creates a batch dataset for the train and test set
    '''
    
    # create a batchdataset for the train set
    train_batch_data = BatchData(Train_Images, Train_Annotations)
            
    # create a batchdataset for the test set
    test_batch_data = BatchData(Test_Images, Test_Annotations)
            
    # return the train and test batch dataset
    return train_batch_data, test_batch_data 

In [ ]:
# The UNet network
def UNet(image, drop_prob = 0.1, init_filters = 64, is_train = True):
    '''This function defines the original UNet network'''
    
    # initialization of the weights
    W_init = tf.contrib.layers.xavier_initializer()
        
    # Unet network
                
    # LEFT part
    # input layer
    input_image = tl.layers.InputLayer(image)
            
    # Convolutional block 1
    conv2d_1 = tl.layers.Conv2d(input_image, init_filters, (3, 3), act= tf.nn.relu, W_init= W_init)
    conv2d_2 = tl.layers.Conv2d(conv2d_1, init_filters, (3, 3), act= tf.nn.relu, W_init= W_init)
    pool_1 = tl.layers.MaxPool2d(conv2d_2, (2, 2), (2, 2), name= 'maxpool_1')
    dropout_1 = tl.layers.DropoutLayer(pool_1, keep= 1-drop_prob, is_fix = True, is_train=is_train)
                
    # Convolutional block 2
    conv2d_3 = tl.layers.Conv2d(dropout_1, 2*init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
    conv2d_4 = tl.layers.Conv2d(conv2d_3, 2*init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
    pool_2 = tl.layers.MaxPool2d(conv2d_4, (2,2), (2, 2), name='maxpool_2')
    dropout_2 = tl.layers.DropoutLayer(pool_2, keep= 1-drop_prob, is_fix = True, is_train=is_train)
                
    # Convolutional block 3
    conv2d_5 = tl.layers.Conv2d(dropout_2, 4*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    conv2d_6 = tl.layers.Conv2d(conv2d_5, 4*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    pool_3 = tl.layers.MaxPool2d(conv2d_6, (2,2), (2, 2), name='maxpool_3')
    dropout_3 = tl.layers.DropoutLayer(pool_3, keep= 1-drop_prob, is_fix = True, is_train=is_train)
                
    # Convolutional block 4
    conv2d_7 = tl.layers.Conv2d(dropout_3, 8*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    conv2d_8 = tl.layers.Conv2d(conv2d_7, 8*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    pool_4 = tl.layers.MaxPool2d(conv2d_8, (2,2), (2, 2), name='maxpool_4')
    dropout_4 = tl.layers.DropoutLayer(pool_4, keep= 1-drop_prob, is_fix = True, is_train=is_train)
                
    # MIDDLE part
    conv2d_9 = tl.layers.Conv2d(dropout_4, 16*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    conv2d_10 = tl.layers.Conv2d(conv2d_9, 16*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
                
                
    # RIGHT part
    # Convolutional block 1
    upsampling_1 = tl.layers.UpSampling2dLayer(conv2d_10, (2,2))
    concat_1 = tl.layers.ConcatLayer([upsampling_1, conv2d_8], 3)
    dropout_5 = tl.layers.DropoutLayer(concat_1, keep= 1-drop_prob, is_fix = True, is_train=is_train)
    conv2d_11 = tl.layers.Conv2d(dropout_5, 8*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    conv2d_12 = tl.layers.Conv2d(conv2d_11, 8*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
                
    # Convolutional block 2
    upsampling_2 = tl.layers.UpSampling2dLayer(conv2d_12, (2,2))
    concat_2 = tl.layers.ConcatLayer([upsampling_2, conv2d_6], 3)
    dropout_6 = tl.layers.DropoutLayer(concat_2, keep= 1-drop_prob, is_fix = True, is_train=is_train)
    conv2d_13 = tl.layers.Conv2d(dropout_6, 4*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    conv2d_14 = tl.layers.Conv2d(conv2d_13, 4*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
            
                
    # Convolutional block 3
    upsampling_3 = tl.layers.UpSampling2dLayer(conv2d_14, (2,2))
    concat_3 = tl.layers.ConcatLayer([upsampling_3,conv2d_4], 3)
    dropout_7 = tl.layers.DropoutLayer(concat_3, keep= 1-drop_prob, is_fix = True, is_train=is_train)
    conv2d_15 = tl.layers.Conv2d(dropout_7, 2*init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
    conv2d_16 = tl.layers.Conv2d(conv2d_15, 2*init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
                
    # Convolutional block 4
    upsampling_4 = tl.layers.UpSampling2dLayer(conv2d_16, (2,2))
    concat_4 = tl.layers.ConcatLayer([upsampling_4,conv2d_2], 3)
    dropout_8 = tl.layers.DropoutLayer(concat_4, keep= 1-drop_prob, is_fix = True, is_train=is_train)
    conv2d_17 = tl.layers.Conv2d(dropout_8, init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
    conv2d_18 = tl.layers.Conv2d(conv2d_17, init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
                
                
    # ouput layer
    output_image = tl.layers.Conv2d(conv2d_18, 1, (1,1), W_init= W_init) 

    # logits tensor, often a step inbetween befor a softmax activation is applied (size im_size x im_size, 2)
    logits = output_image.outputs

    # the model
    model = output_image

    # a binary output map with size im_size x im_size can be found by applying the argmax operation
    # this operation yields 0 or 1 for every pixel position depending on whether the value in the first or second image is the largest
    binary_prediction = tf.math.argmax(logits, axis= 3, name= 'prediction')
        
    return binary_prediction, logits, output_image

In [ ]:
# define some different losses
def loss_UNet(predicted_logits, real_annotations):

    real_annotations = tf.convert_to_tensor(real_annotations)
    real_annotations = tf.cast(real_annotations, tf.float32)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels= real_annotations, logits= predicted_logits)
    return tf.reduce_mean(loss)

# define the focal cross entropy loss function
# loss function has to be defined in a way to avoid data inbalance
def loss_sfce(predicted_logits, real_annotations, alpha = 0.25, gamma = 2.0):
    '''
    This type of loss function tries to avoid data imbalance in image segmentation
    There are two parameters alpha and gamma, the default values are indicated
    gamma should always be greater than or equal to 0
    '''
    print('calculating loss')
    
    predicted_logits = tf.convert_to_tensor(predicted_logits)
    real_annotations = tf.convert_to_tensor(real_annotations)
    real_annotations = tf.cast(real_annotations, tf.float32)
    
    # classic binary cross_entropy is calculated
    ce = K.binary_crossentropy(real_annotations, predicted_logits, from_logits= True)

    # binary cross-entropy is multiplied with two factors: alpha and modulating factor
    # convert the logits predictions into probabilities
    alpha_factor = 1.0
    modulating_factor = 1.0
    
    pred_prob = tf.sigmoid(predicted_logits)
    
    if alpha:
        alpha = tf.convert_to_tensor(alpha, dtype=K.floatx())
        alpha_factor = real_annotations * alpha + (1 - real_annotations) * (1 - alpha)


    p_t = (real_annotations * pred_prob) + ((1 - real_annotations) * (1 - pred_prob))
    if gamma:
        gamma = tf.convert_to_tensor(gamma, dtype=K.floatx())
        modulating_factor = tf.pow((1.0 - p_t), gamma)

    # compute the final loss and return
    return tf.reduce_mean(alpha_factor * modulating_factor * ce)



# Asymmetric similarity loss function, to balance recall and precision
# the larger beta, the more important the recall becomes relative to the precision
def loss_asl(predicted_logits, real_annotations, beta = 2):
    real_annotations = real_annotations[:,:,:,0]
    real_annotations = tf.cast(real_annotations, tf.float32)
    pred_prob = tf.nn.sigmoid(predicted_logits)[:,:,:,0]
    
    prod_pos = pred_prob * real_annotations
    sum_prod_pos = tf.reduce_sum(tf.reduce_sum(prod_pos, axis = 2), axis = 1)
    prod_neg_pred = (1-pred_prob) * real_annotations
    sum_prod_neg_pred = tf.reduce_sum(tf.reduce_sum(prod_neg_pred, axis = 2), axis = 1)
    prod_neg_real = (pred_prob) * (1-real_annotations)
    sum_prod_neg_real = tf.reduce_sum(tf.reduce_sum(prod_neg_real, axis = 2), axis = 1)

    beta = tf.convert_to_tensor(beta, dtype=K.floatx())

    num = (1+beta**2) * sum_prod_pos
    denom = (1+beta**2) *sum_prod_pos + beta**2 * sum_prod_neg_pred + sum_prod_neg_real

    loss = num/denom
    return tf.reduce_mean(loss)

In [ ]:
# define some optimizers and apply them to the network
def Adam_optimization(losses, lr = 0.001):
    '''This function defines the Adam optimizer that will be used and applies it to optimize the weights during training'''
    optimizer = tf.train.AdamOptimizer(learning_rate = lr)
    train_variables_list = tf.trainable_variables()
    gradients = optimizer.compute_gradients(losses, var_list= train_variables_list)
    train_optimization = optimizer.apply_gradients(gradients)
            
    return train_optimization

def SGD_optimization(losses, lr = 0.001):
    '''This function defines the Stochastic gradient descent optimizer that will be used and applies it to optimize the weights during trainin'''
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
    train_variables_list = tf.trainable_variables()
    gradients = optimizer.compute_gradients(losses, var_list= train_variables_list)
    train_optimization = optimizer.apply_gradients(gradients)

    return train_optimization

In [ ]:
def train_network(TrainImages, TrainAnnotations, TestImages, TestAnnotations, 
                  Drop_Prob = 0.1, Init_Filters = 64, batch_size = 3, loss_function = 'UNet_loss', optim = 'Adam', 
                  learning_rate = tf.Variable(1e-5, dtype=tf.float32), MAX_EPOCH = 10, SaveResults = True, print_freq = 1):
    '''
    This function trains the UNet on the indicated train data with corresponding annotations
    At the end the trained model is being saved
    '''

    # placeholders are created, variables to which data is assigned later on
    print('Create placeholders')
    image = tf.placeholder(tf.float32, [None,512,512, 3], name= 'image')
    annotation = tf.placeholder(tf.int32, shape=[None, 512, 512, 1], name= "annotation")

    # define the model that will be used for training and for testing
    print('Define the model for training')
    train_pred, train_logits, train_network = UNet(image, drop_prob = Drop_Prob, init_filters = Init_Filters, is_train = True)
    print('Define the model for testing')
    test_pred, test_logits, test_network = UNet(image, drop_prob = Drop_Prob, init_filters = Init_Filters, is_train = False)
    
    # define the output probability maps, losses and optimization for training and testing
    print('Define outputs, losses and optimization')
    # sigmoid activation creates a probability map which is the output
    train_positive_prob = tf.nn.sigmoid(train_logits)
    test_positive_prob = tf.nn.sigmoid(test_logits)
    # loss function
    if loss_function == 'UNet_loss':
        train_loss_op = loss_UNet(train_logits, annotation)
        test_loss_op= loss_UNet(test_logits, annotation)
    elif loss_function == 'Sfce_loss':
        train_loss_op = loss_sfce(train_logits, annotation, alpha = 0.25, gamma = 2.0)
        test_loss_op= loss_sfce(test_logits, annotation, alpha = 0.25, gamma = 2.0)
    elif loss_function == 'Asl_loss':
        train_loss_op = loss_asl(train_logits, annotation)
        test_loss_op= loss_asl(test_logits, annotation)
    # optimization
    if optim == 'Adam':
        train_op = Adam_optimization(train_loss_op, learning_rate)
    elif optim == 'sgd':
        train_op = SGD_optimization(train_loss_op, learning_rate)
        
    # learning_rate decay at higher amounts of epochs 
    lr_assign_op = tf.assign(learning_rate, learning_rate / 10)
    

    # a session should be started in which all intermediate results are stored (also weights of the network for example)
    sess = tf.Session()
    # saving the train and test results and the trained model
    if SaveResults:
        # creating summary which stores the results that can be visualised with tensorboard
        print("Setting up summary...")
        # test_summary_op = tf.summary.merge_all()
        # creating saver, used to save the trained model at the end
        print("Setting up Saver...")
        saver = tf.train.Saver(max_to_keep=2)
        summary_writer = tf.summary.FileWriter(log_dir_tens, sess.graph)

        
    tl.layers.initialize_global_variables(sess)
    sess.run(tf.global_variables_initializer())
    
    # define the train and test batches that can be fed into the network
    train_batch_data, test_batch_data = LoadBatchData(TrainImages, TrainAnnotations, TestImages, TestAnnotations)

    # defining the numer of steps per epoch, based on the batchsize
    train_nbr = TrainImages.shape[0]
    test_nbr = TestImages.shape[0]
    step_every_epoch = int(train_nbr/batch_size)
    test_every_epoch = int(test_nbr/batch_size)
    
    # go over all epochs of training
    for epo in range(1,MAX_EPOCH+1):
        
        # time is defined at the start of every epoch
        start_time = time.time()
        # resetting of all variables
        train_loss, test_loss, train_aupr, test_aupr, train_auc, test_auc= 0, 0, 0, 0, 0, 0
            
        # go over all batches in one epoch
        for s in range(step_every_epoch):
        
            # define the next batch to train the network
            train_images_batch, train_annotations_batch = train_batch_data.next_batch(batch_size)
            feed_dict = {image: train_images_batch, annotation: train_annotations_batch}
            # train the network and define the output of the network for this batch of images
            train_pos_prob, train_err, _ = sess.run([train_positive_prob, train_loss_op, train_op], feed_dict=feed_dict)

            # auc and aupr score are calculated for this batch
            temp_train_annotations = np.reshape(train_annotations_batch,-1)
            temp_tra_positive_prob = np.reshape(train_pos_prob,-1)
            train_sauc = ROC(temp_tra_positive_prob, temp_train_annotations, plot = False)
            train_saupr = PRC(temp_tra_positive_prob, temp_train_annotations, plot = False)

            # define the total loss, auc and aupr over all batches in 1 epoch
            train_loss += train_err
            train_auc += train_sauc
            train_aupr += train_saupr
            
        # after every epoch, check whether results should be printed out and network should be tested
        # print out after epoch 1 and then every print_freq
        if epo % print_freq == 0 or epo == 1 or epo == (MAX_EPOCH):
        
            # average out loss, auc and aupr for the train set over all batches
            train_loss = train_loss/step_every_epoch
            train_auc = train_auc/step_every_epoch
            train_aupr = train_aupr/step_every_epoch

            # print out the training results
            print('epoch {} took {}s'.format(epo, time.time() - start_time))
            print('   train loss: {}'.format(train_loss))
            print('   train auc: {}'.format(train_auc))
            print('   train aupr: {}'.format(train_aupr))

            if SaveResults:      
                # save these values to visualize them later with tensorboard
                train_summary = tf.Summary(value=[tf.Summary.Value(tag="train_loss", simple_value=train_loss), tf.Summary.Value(tag="train_auc", simple_value=train_auc),
                                                tf.Summary.Value(tag="train_aupr", simple_value=train_aupr)])
                summary_writer.add_summary(train_summary, epo)
                
            # testing of the network
            # test data is also subdivided in batches, go over all batches
            for test_s in range(test_every_epoch):

                # get the next batch of test data
                test_images_batch, test_annotations_batch = test_batch_data.next_batch(batch_size)
                # apply the network to the test images and define the output of the network and the loss
                feed_dict= {image:test_images_batch, annotation:test_annotations_batch}
                test_pos_prob, test_err = sess.run([test_positive_prob, test_loss_op], feed_dict= feed_dict)

                # compute auc and aupr score for test set
                temp_test_annotations = np.reshape(test_annotations_batch,-1)
                temp_test_positive_prob = np.reshape(test_pos_prob,-1)
                test_sauc = ROC(temp_test_positive_prob, temp_test_annotations, plot = False)
                test_saupr= PRC(temp_test_positive_prob, temp_test_annotations, plot = False)

                # calculate total loss, auc and aupr for the test set over all batches
                test_loss += test_err
                test_auc += test_sauc
                test_aupr += test_saupr

            # average loss, auc and aupr for the test set over all batches
            test_loss = test_loss/test_every_epoch
            test_auc = test_auc/test_every_epoch
            test_aupr = test_aupr/test_every_epoch

            # print out the test results
            print('   test loss: {}'.format(test_loss))
            print('   test auc: {}'.format(test_auc))
            print('   test aupr: {}'.format(test_aupr))

            if SaveResults:       
                # save these values to visualize them later with tensorboard
                test_summary = tf.Summary(value=[tf.Summary.Value(tag="test_loss", simple_value=test_loss), tf.Summary.Value(tag="test_auc", simple_value=test_auc), 
                                                tf.Summary.Value(tag="test_aupr", simple_value=test_aupr)])
                summary_writer.add_summary(test_summary, epo) 

                # tensorboard flush
                summary_writer.flush() # the summary is written at this moment
            
        # at specific amounts of epochs, the learning rate should become smaller to work more precisely
        if epo == int(MAX_EPOCH*2/3) or epo == int(MAX_EPOCH/2): 
            sess.run(lr_assign_op)
            
        # eventual saving of the fully trained model
        if SaveResults: 
            if epo == MAX_EPOCH:
                saver.save(sess, log_dir_model + "model.ckpt", epo)
                print('epoch {}, the model has been saved successfully'.format(epo))

    return test_loss, test_auc, test_aupr   

In [ ]:
def ROC(pred_probs, true_probs, plot = True):
    '''
    This function calculates the ROC-AUC value
    and it also calculates and visualizes the ROC-curve (if plot is true)
    '''
    
    # calculate and print out ROC-AUC value
    ROC_AUC = roc_auc_score(true_probs, pred_probs)
    
    # calculate and plot the ROC-curve
    if plot:
        FPRate, TPRate, Thresh = roc_curve(true_probs, pred_probs)
        plt.figure()
        plt.plot(FPRate, TPRate)
        plt.title('ROC curve')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.show()

    return ROC_AUC

In [ ]:
def PRC(pred_probs, true_probs, plot = True):
    '''
    Calculate the aupr value = the area under the precision-recall curve
    and plot the precision-recall curve (if plot is true)
    '''
    
    # calculate the precision-recall curve
    Precision, Recall, Thresh = precision_recall_curve(true_probs, pred_probs)
    Precision = np.fliplr([Precision])[0]  # so the array is increasing (you won't get negative AUC)
    Recall = np.fliplr([Recall])[0]  # so the array is increasing (you won't get negative AUC)
    AUPR = np.trapz(Precision, Recall)
      
    
    # plot the precision-recall curve
    if plot:
        plt.figure()
        plt.plot(Precision, Recall)
        plt.title('Precision-Recall curve')
        plt.xlabel('Precision')
        plt.ylabel('Recall')
        plt.show()
    
    return AUPR

In [ ]:
train_network(train_images, train_annotations, test_images, test_annotations, loss_function = 'Sfce_loss', MAX_EPOCH = 20)
# TrainImages, TrainAnnotations, TestImages, TestAnnotations, 
#                   Drop_Prob = 0.1, Init_Filters = 64, batch_size = 3, loss_function = 'UNet_loss', optim = 'Adam', 
#                   learning_rate = tf.Variable(1e-3, dtype=tf.float32), MAX_EPOCH = 10, SaveResults = True

Create placeholders
Define the model for training
[TL] InputLayer  input: (?, 512, 512, 3)
[TL] Conv2d conv2d: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv2d: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d maxpool_1: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] DropoutLayer dropout_layer: keep: 0.900000 is_fix: True
[TL] Conv2d conv2d: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv2d: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d maxpool_2: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] DropoutLayer dropout_layer: keep: 0.900000 is_fix: True
[TL] Conv2d conv2d: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv2d: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d maxpool_3: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] DropoutLayer dro

(0.12864714459731028, 0.5018781512049466, 0.027091534842790144)